In [1]:
import xesmf
import xarray as xr
import os
import cmocean as cm
import cartopy.crs as ccrs
import cosima_cookbook as cc
from dask.distributed import Client
import dask.array as da

In [2]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36153,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:41559,Total threads: 4
Dashboard: /proxy/45055/status,Memory: 35.88 GiB
Nanny: tcp://127.0.0.1:46767,


In [3]:
GEBCO_data = xr.open_dataset('/g/data/ik11/inputs/GEBCO_2024/GEBCO_2024.nc', decode_times=False)

In [11]:
Bed_machine_data = xr.open_dataset('/g/data1a/tm70/ek4684/Bedmachine_data/Updated_BedMachineAntarctica_2020-07-15_v02.nc', decode_times=False).chunk(chunks={'x': 1000, 'y': 1000})

In [5]:
topog = xr.open_dataset('/g/data/vk83/prerelease/configurations/inputs/access-om3/share/grids/2025.02.17/topog.nc', decode_times=False).rename({'nx': 'x', 'ny': 'y', 'geolon_t': 'lon', 'geolat_t': 'lat'}).chunk(chunks={'x': 100, 'y': 100})

In [6]:
topog.chunks

Frozen({'y': (100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 42), 'x': (100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 40)})

In [18]:
topog

<xarray.Dataset> Size: 26MB
Dimensions:            (y: 1142, x: 1440)
Dimensions without coordinates: y, x
Data variables:
    depth              (y, x) float32 7MB dask.array<chunksize=(100, 100), meta=np.ndarray>
    sea_area_fraction  (y, x) float32 7MB dask.array<chunksize=(100, 100), meta=np.ndarray>
    lon                (y, x) float32 7MB dask.array<chunksize=(100, 100), meta=np.ndarray>
    lat                (y, x) float32 7MB dask.array<chunksize=(100, 100), meta=np.ndarray>
Attributes:
    original_file:  topog_new_fillfraction_edited_deseas.nc
    input_file:     /g/data/vk83/prerelease/configurations/inputs/access-om3/...
    history:        Sun Feb 17 14:12:24 2025 +1100: topogtools min_max_depth ...

In [12]:
Bed_machine_data.chunks

Frozen({'y': (1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 333), 'x': (1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 333)})

In [15]:
Bed_machine_data

<xarray.Dataset> Size: 5GB
Dimensions:    (y: 13333, x: 13333)
Dimensions without coordinates: y, x
Data variables:
    mapping    |S1 1B ...
    mask       (y, x) int8 178MB dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    firn       (y, x) float32 711MB dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    surface    (y, x) float32 711MB dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    thickness  (y, x) float32 711MB dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    bed        (y, x) float32 711MB dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    errbed     (y, x) int16 356MB dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    source     (y, x) int8 178MB dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    geoid      (y, x) int16 356MB dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    lat        (y, x) float32 711MB dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    lon        (y, x) float32 711MB dask.array<chunksize=(1000, 1000), meta=np.ndarray>

In [19]:
regridder_Bed_machine_to_GEBCO = xesmf.Regridder(
    Bed_machine_data,
    topog,
    "bilinear",
    periodic=True,
    # parallel=True,
    # filename='/g/data1a/tm70/ek4684/Bedmachine_data/Regridd_Bedmachine_to_GEBCO_Bilinear.nc')
    filename='/g/data/v45/aek156/notebooks/github/ezhilsabareesh8/Notebooks/Regridd_Bedmachine_to_GEBCO_Bilinear.nc')